In [1]:
cd ../..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import (Dense, Dropout, Flatten, Input, BatchNormalization, Lambda, 
                                     concatenate, Embedding, Reshape)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback, LearningRateScheduler

# import wandb
# from wandb.keras import WandbCallback

# own imports
from evaluation import Referee, load_data, select_dates, select_day_nums
from train import (BatchCreator, Logger, plot_confidence_series, plot_some_confidence_intervals, 
                   prepare_training, perform_training_scheme)
from model_builder import (get_pinball_losses, get_simple_dense_model, get_simple_dist_model,
                           get_variable_dist_model, get_direct_dist_model)
from flow import (model_predict, denorm_preds, warp_preds_to_ref_form, plot_some, evaluate_model, 
                  save_object, load_object)
from preprocess import (preprocess, get_features, pandas_cat_data, reset_categorical_features, 
                        read_and_preprocess_data)


# Setup file paths
data_dir = 'data/'
os.environ['DATA_DIR'] = data_dir
sub_dir = 'submissions_uncertainty/'
plt.rcParams['font.size'] = 13

# Hardcode requested quantiles
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]

/Users/joeranbosma/opt/anaconda3/envs/cart/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/joeranbosma/opt/anaconda3/envs/cart/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/joeranbosma/opt/anaconda3/envs/cart/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/joeranbosma/opt

In [3]:
def run_experiment(nodes_settings, mode="dense", augment_events=False, 
                   lgb_prediction=False, lgb_prediction_dir=None):
    """
    Run experiment on the performance of the final "distribution layer"
    nodes_settings: number of nodes per layer to test
                    should be in the format {level: [n1, n2, ...], ...}
    mode: distribution layer to test, either "dense", "dist2", "dist4" or "direct"
    """
    
    # setup model builder function
    if mode == "dense":
        model_func = get_simple_dense_model
    elif mode == "dist2":
        model_func = get_simple_dist_model
    elif mode == "dist4":
        model_func = get_variable_dist_model
    elif mode == "direct":
        model_func = get_direct_dist_model
    
    # track metrics
    logger_list = []
    part1_metrics = []
    part2_metrics = []
    part3_metrics = []
    
    # loop over the levels
    for level, node_options in nodes_settings.items():
        
        # loop over the number of nodes per layer
        for num_nodes in node_options:
            model_name = "model_{}_{}".format(mode, num_nodes)
            
            # build model
            model = model_func(inp_shape=input_shapes[level], num_nodes=num_nodes, final_activation="exponential")
            model.summary()
            
            # train model
            warmup_lr_list = [1e-3, 1e-3, 1e-3, # save part 1
                              1e-3, 1e-3, 1e-3] # save part 2
            finetune_lr_list = [1e-4, 1e-4, 1e-5, 1e-5]
            model, logger, metrics1, metrics2, metrics3 = perform_training_scheme(
                level, model, warmup_batch_size, finetune_batch_size, ref, calendar,
                model_dir=model_dir, model_name=model_name, warmup_lr_list=warmup_lr_list, 
                finetune_lr_list=finetune_lr_list, lgb_prediction=lgb_prediction,
                lgb_prediction_dir=lgb_prediction_dir, augment_events=augment_events,
            )
            
            # save metrics
            logger_list.append(logger); part1_metrics.append(metrics1); 
            part2_metrics.append(metrics2); part3_metrics.append(metrics3)
            for i, m in enumerate([part1_metrics, part2_metrics, part3_metrics]):
                save_object(m, model_dir + mode + "_part{}_metrics.pickle".format(i+1))
            
            # save training metrics
            save_object(logger.train_metrics, model_dir + model_name + "_metrics_train.pickle")
            save_object(logger.val_metrics, model_dir + model_name + "_metrics_val.pickle")

# Train MLP - multiple levels

In [4]:
# Load in the data set
calendar, sales_train_validation, sell_prices = load_data()

sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
sales_train = select_dates(sales_train_validation, day_start=1, num_days=1913-28, include_metadata=True)

ref = Referee(sales_true, sales_train, sell_prices, calendar, verbose=True)

Initializing Referee
Converting true sales to quantile form
Calculating weights for each level...
Calculating scale for each level...
Finished setup.


In [11]:
model_dir = 'models/uncertainty/experiment/lgb-events-run1/'
lgb_prediction_dir = 'models/uncertainty/train_lightgbm_events/run1/'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

## Define model architectures

In [8]:
labels = ['demand']

# Set input sizes for the model
input_shapes = {
    1: (178,),
    2: (181,),
    3: (191,),
    4: (181,),
    5: (188,),
    6: (184,),
    7: (191,),
    8: (194,),
    9: (201,),
    10: (3237,),
    11: (3240,),
#     12: (3248,),
}

augment_events = True
lgb_prediction = True
if lgb_prediction:
    # add one to the input shape
    for key, val in input_shapes.items():
        input_shapes[key] = (val[0]+1,)

# Set initial batch size
warmup_batch_size = {level: 64 if level <= 9 else 1024 for level in range(1, 1+12)}

# Set fine-tune batch size
finetune_batch_size = {level: 256 if level <= 9 else 25000 for level in range(1, 1+12)}

## Define experiment settings

In [9]:
def get_nodes_settings(mode):
    nodes_dense_small = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
    nodes_dist2_small = [1, 2, 4, 6, 8, 16, 32, 64, 128, 256]
    nodes_dist4_small = [1, 2, 4, 6, 8, 16, 32, 64, 128, 256]
    nodes_direct_small = [1, 2, 4, 6, 8, 16, 32, 64, 128, 256]
    
    # setup node settings
    if mode == "dense":
        nodes_settings = {level: nodes_dense_small for level in range(1, 9+1)}
    elif mode == "dist2":
        nodes_settings = {level: nodes_dist2_small for level in range(1, 9+1)}
    elif mode == "dist4":
        nodes_settings = {level: nodes_dist4_small for level in range(1, 9+1)}
    elif mode == "direct":
        nodes_settings = {level: nodes_direct_small for level in range(1, 9+1)}

    return nodes_settings

In [12]:
for mode in ["direct", "dist2", "dist4", "dense"]:
    print("Starting mode {}..".format(mode))
    
    nodes_settings = get_nodes_settings(mode)
    print(nodes_settings)
    
    run_experiment(nodes_settings, mode=mode, augment_events=augment_events,
                   lgb_prediction=lgb_prediction, lgb_prediction_dir=lgb_prediction_dir)

Starting mode direct..
{1: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 2: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 3: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 4: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 5: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 6: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 7: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 8: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256], 9: [1, 2, 4, 6, 8, 16, 32, 64, 128, 256]}
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 179)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            180         input[0][0]                      
__________________________________________________________________________________________________
dens

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Adding days prior/after events to calendar..
Mem. usage decreased to  0.00 Mb (74.3% reduction)
1 28
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 1913 rows and 3 columns
Selecting 2190 rows (114.5%)
Mem. usage decreased to  0.15 Mb (0.0% reduction)
Saving to file..
Finished.
Converting to pandas data..
Resetting categories..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t28', 'shift_t29', 'shift_t30', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'm

TypeError: Unexpected keyword argument passed to optimizer: learning_rate

## Test reading